In [17]:
# Import necessary libraries
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import ee
from google.oauth2 import service_account
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Initialize Earth Engine API
service_account_key = r'C:\Users\serafino\Desktop\Water Management System\ee-eugenioserafino01-dca7dda5348b.json'  # Replace with your service account key path

# Define the required OAuth scope for Earth Engine
scopes = ['https://www.googleapis.com/auth/earthengine']

# Create credentials with the specified scope
credentials = service_account.Credentials.from_service_account_file(
    service_account_key, scopes=scopes
)

# Initialize Earth Engine with the credentials
ee.Initialize(credentials=credentials)

# Define the coordinates of your area of interest
pontelongo_coordinates = [
    [12.021280, 45.251061],
    [12.021719, 45.250949],
    [12.022229, 45.251746],
    [12.022891, 45.251549],
    [12.023380, 45.252315],
    [12.023846, 45.252183],
    [12.024085, 45.252617],
    [12.022100, 45.252718],
    [12.021280, 45.251061]  # Closing the polygon by repeating the first point
]

days_before_today = 500
forecast_days = 3


In [18]:
def get_sentinel2_data(polygon_coordinates, days_before_today, collection_type='COPERNICUS/S2', cloud_percentage=None, aggregate_by_month=False):
    """
    Fetches NDVI, NDMI, and SWIR time series for the given polygon over the specified date range,
    with options to filter by cloud cover, choose collection type, and aggregate by month.

    Parameters:
    - polygon_coordinates: List of [longitude, latitude] coordinates defining the polygon.
    - days_before_today: Number of days before today to start the data collection.
    - collection_type: Sentinel-2 collection type ('COPERNICUS/S2' for Level-1C, 'COPERNICUS/S2_HARMONIZED' for Level-2A).
    - cloud_percentage: Maximum cloud cover percentage to allow (default is None, no cloud filter).
    - aggregate_by_month: Boolean to indicate if data should be aggregated by month.

    Returns:
    - DataFrame containing dates, NDVI, NDMI, and SWIR statistics (mean).
    """
    # Calculate date range
    end_date = datetime.today()
    start_date = end_date - timedelta(days=days_before_today)
    end_date_str = end_date.strftime('%Y-%m-%d')
    start_date_str = start_date.strftime('%Y-%m-%d')

    # Define the polygon geometry
    location_polygon = ee.Geometry.Polygon([polygon_coordinates])

    # Load Sentinel-2 image collection (Level-1C or Level-2A)
    collection = ee.ImageCollection(collection_type) \
                   .filterDate(start_date_str, end_date_str) \
                   .filterBounds(location_polygon)

    # Apply cloud filter if specified
    if cloud_percentage is not None:
        collection = collection.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percentage))

    # Print the number of images retrieved for debugging
    print("Number of images retrieved:", collection.size().getInfo())

    # Function to calculate indices
    def calculate_indices(image):
        ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
        ndmi = image.normalizedDifference(['B8', 'B11']).rename('NDMI')
        swir = image.select(['B11']).rename('SWIR')
        return image.addBands([ndvi, ndmi, swir])

    # Map the function to the collection
    collection_with_indices = collection.map(calculate_indices)

    # Reduce by region to get mean values for each day
    def get_stats_indices(image):
        stats_dict = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=location_polygon,
            scale=10,
            maxPixels=1e9
        )
        stats_dict = stats_dict.set('date', image.date().format('YYYY-MM-dd'))
        return ee.Feature(None, stats_dict)

    # Apply to each image and convert to a FeatureCollection
    stats_indices_fc = ee.FeatureCollection(collection_with_indices.map(get_stats_indices))

    # Convert to a list and fetch data
    indices_data = stats_indices_fc.getInfo()['features']

    # Parse data into a DataFrame
    dates = []
    ndvi_mean = []
    ndmi_mean = []
    swir_mean = []

    for feature in indices_data:
        properties = feature['properties']
        dates.append(properties['date'])
        ndvi_mean.append(properties.get('NDVI', np.nan))
        ndmi_mean.append(properties.get('NDMI', np.nan))
        swir_mean.append(properties.get('SWIR', np.nan))

    # Create DataFrame
    df = pd.DataFrame({
        'date': pd.to_datetime(dates),
        'NDVI_mean': ndvi_mean,
        'NDMI_mean': ndmi_mean,
        'SWIR_mean': swir_mean
    })

    # Set 'date' as index
    df.set_index('date', inplace=True)

    # Handle missing values by interpolation
    df.sort_index(inplace=True)
    df.interpolate(method='time', inplace=True)
    df.dropna(inplace=True)
    
    # Aggregate by month if specified
    if aggregate_by_month:
        df = df.resample('M').mean()

    df.reset_index(inplace=True)

    return df


sentinel2_df = get_sentinel2_data(pontelongo_coordinates, days_before_today)

# Print the first few rows
print("Sentinel-2 Data:")
print(sentinel2_df.head())


Number of images retrieved: 97
Sentinel-2 Data:
        date  NDVI_mean  NDMI_mean    SWIR_mean
0 2023-07-02   0.117166  -0.003981  3073.815984
1 2023-07-07   0.128470  -0.087528  4183.385317
2 2023-07-12   0.148159  -0.063909  4166.751277
3 2023-07-17   0.145976  -0.094330  4278.344267
4 2023-07-22   0.164082  -0.082508  4089.210707


In [19]:
# Function to retrieve the specified variables and create a DataFrame
def retrieve_soil_moisture_and_related_data(polygon_coordinates, days_before_today=90):
    # Set the date range
    end_date = datetime.today()
    start_date = end_date - timedelta(days=days_before_today)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    # Create the polygon for the area of interest
    location_polygon = ee.Geometry.Polygon([polygon_coordinates])

    # Define the variables to retrieve
    variables = [
        'dewpoint_temperature_2m', 'temperature_2m', 'skin_temperature',
        'soil_temperature_level_1', 'soil_temperature_level_2', 'soil_temperature_level_3', 'soil_temperature_level_4',
        'snow_cover', 'snow_depth', 'skin_reservoir_content', 'volumetric_soil_water_layer_1', 
        'volumetric_soil_water_layer_2', 'volumetric_soil_water_layer_3', 'volumetric_soil_water_layer_4', 
        'forecast_albedo', 'surface_latent_heat_flux_sum', 'surface_net_solar_radiation_sum', 
        'surface_net_thermal_radiation_sum', 'surface_sensible_heat_flux_sum', 'surface_solar_radiation_downwards_sum', 
        'surface_thermal_radiation_downwards_sum', 'evaporation_from_bare_soil_sum', 'evaporation_from_the_top_of_canopy_sum',
        'evaporation_from_vegetation_transpiration_sum', 'potential_evaporation_sum', 'runoff_sum', 
        'sub_surface_runoff_sum', 'surface_runoff_sum', 'total_evaporation_sum', 'total_precipitation_sum', 'leaf_area_index_high_vegetation', 
        'leaf_area_index_low_vegetation'
    ]

    # Load ERA5 Land daily aggregated collection and filter by date and region
    era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR') \
        .filterDate(start_date_str, end_date_str) \
        .filterBounds(location_polygon) \
        .select(variables)

    # Function to extract mean values for each image date and variable
    def get_daily_data(image):
        stats_dict = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=location_polygon,
            scale=1000,
            maxPixels=1e9
        )
        stats_dict = stats_dict.set('date', image.date().format('YYYY-MM-dd'))
        return ee.Feature(None, stats_dict)

    # Apply function to each image in collection
    daily_data = era5_collection.map(get_daily_data)

    # Convert FeatureCollection to list and fetch data
    data = daily_data.getInfo()['features']
    
    # Parse data into a DataFrame
    dates = []
    data_dict = {var: [] for var in variables}
    
    for feature in data:
        properties = feature['properties']
        dates.append(properties['date'])
        for var in variables:
            data_dict[var].append(properties.get(var, None))
    
    # Create DataFrame
    df = pd.DataFrame(data_dict)
    df['date'] = pd.to_datetime(dates)
    df.set_index('date', inplace=True)
    
    # Interpolate any missing data
    df = df.apply(pd.to_numeric, errors='coerce')
    df.interpolate(method='time', inplace=True)
    
    # Return the DataFrame
    return df

df = retrieve_soil_moisture_and_related_data(pontelongo_coordinates, days_before_today=90)

# Print DataFrame
print(df)


            dewpoint_temperature_2m  temperature_2m  skin_temperature  \
date                                                                    
2024-08-11               295.192231      301.765735        302.207139   
2024-08-12               295.057742      302.852717        303.040889   
2024-08-13               296.265884      302.787551        303.332196   
2024-08-14               295.844050      301.962976        302.833584   
2024-08-15               293.803665      301.738520        302.760049   
...                             ...             ...               ...   
2024-10-29               287.382231      289.033012        289.361224   
2024-10-30               286.690476      289.132823        288.654848   
2024-10-31               286.166370      288.745216        287.705943   
2024-11-01               285.779329      286.790503        286.950136   
2024-11-02               285.693310      286.926168        287.410834   

            soil_temperature_level_1  soil_tempera

In [20]:
def get_soil_moisture_data(polygon_coordinates, days_before_today, collection_id='NASA/SMAP/SPL4SMGP/007', band_name='sm_rootzone'):
    """
    Fetches soil moisture data from Earth Engine in quarterly increments over a specified polygon.
    Parameters:
        polygon_coordinates (list): List of [lon, lat] coordinates defining a polygon.
        days_before_today (int): Number of days before today to start data retrieval.
        collection_id (str): Earth Engine collection ID for soil moisture data.
        band_name (str): Band name for soil moisture data in the collection.
    Returns:
        pd.DataFrame: DataFrame with 'date' as index and 'soil_moisture' values.
    """
    polygon = ee.Geometry.Polygon([polygon_coordinates])
    end_date = ee.Date(datetime.now().strftime('%Y-%m-%d'))
    start_date = end_date.advance(-days_before_today, 'day')

    results = []
    current_date = start_date
    while current_date.difference(end_date, 'month').getInfo() < 0:
        next_date = current_date.advance(3, 'month')
        dataset = ee.ImageCollection(collection_id).filterDate(current_date, next_date).select(band_name)
        quarterly_data = dataset.map(lambda image: ee.Feature(
            None,
            {
                'date': image.date().format('YYYY-MM-dd'),
                'soil_moisture': image.reduceRegion(
                    reducer=ee.Reducer.mean(),
                    geometry=polygon,
                    scale=1000
                ).get(band_name)
            }
        ))

        try:
            quarterly_data_list = quarterly_data.getInfo()['features']
            results.extend([{
                'date': feature['properties']['date'],
                'soil_moisture': feature['properties'].get('soil_moisture', None)
            } for feature in quarterly_data_list])
        except Exception as e:
            print(f"Error retrieving data for {current_date.format('YYYY-MM').getInfo()}: {e}")
        
        current_date = next_date

    soil_moisture_df = pd.DataFrame(results)
    soil_moisture_df['date'] = pd.to_datetime(soil_moisture_df['date'])
    soil_moisture_df.set_index('date', inplace=True)
    
    # Interpolate missing values if any
    soil_moisture_df.interpolate(method='time', inplace=True)
    
    return soil_moisture_df


def get_lst_data(polygon_coordinates, days_before_today):
    """
    Fetches Land Surface Temperature (LST) data over the specified date range.
    Parameters:
        polygon_coordinates (list): List of [lon, lat] coordinates defining a polygon.
        days_before_today (int): Number of days before today to start data retrieval.
    Returns:
        pd.DataFrame: DataFrame with 'date' as index and 'lst' values in Celsius.
    """
    # Calculate date range
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_before_today)
    end_date_str = end_date.strftime('%Y-%m-%d')
    start_date_str = start_date.strftime('%Y-%m-%d')
    
    # Create a polygon geometry for the specified coordinates
    polygon = ee.Geometry.Polygon([polygon_coordinates])
    
    # Load MODIS LST collection and filter by date range
    lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
        .filterDate(start_date_str, end_date_str) \
        .select('LST_Day_1km')

    # Function to get LST
    def get_lst(image):
        lst = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=polygon,
            scale=1000,
            maxPixels=1e9
        )
        date = image.date().format('YYYY-MM-dd')
        return ee.Feature(None, lst.set('date', date))

    # Apply to each image and convert to a FeatureCollection
    lst_fc = lst_collection.map(get_lst)
    lst_data = lst_fc.getInfo()['features']

    # Parse data into a DataFrame
    dates = []
    lst_values = []
    for feature in lst_data:
        properties = feature['properties']
        dates.append(properties['date'])
        lst_values.append(properties.get('LST_Day_1km', np.nan))

    df = pd.DataFrame({
        'date': pd.to_datetime(dates),
        'lst': lst_values
    })

    # Convert LST from Kelvin*0.02 to Celsius
    df['lst'] = df['lst'] * 0.02 - 273.15

    # Set 'date' as index
    df.set_index('date', inplace=True)

    # Handle any missing values
    df.sort_index(inplace=True)
    df.interpolate(method='time', inplace=True)
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    return df


# Fetch Soil Moisture Data
soil_moisture_df = get_soil_moisture_data(pontelongo_coordinates, days_before_today)
print("Soil Moisture Data:")
print(soil_moisture_df)

# Fetch LST Data
lst_df = get_lst_data(pontelongo_coordinates, days_before_today)
print("LST Data:")
print(lst_df)

Soil Moisture Data:
            soil_moisture
date                     
2023-06-28       0.169126
2023-06-28       0.169567
2023-06-28       0.170509
2023-06-28       0.169843
2023-06-28       0.168879
...                   ...
2024-11-06       0.276822
2024-11-06       0.276453
2024-11-06       0.276309
2024-11-06       0.276303
2024-11-06       0.276341

[3978 rows x 1 columns]
LST Data:
          date     lst
0   2023-06-29  31.610
1   2023-06-30  31.826
2   2023-07-01  32.042
3   2023-07-02  32.258
4   2023-07-03  32.474
..         ...     ...
492 2024-11-02  18.030
493 2024-11-03  16.820
494 2024-11-04  15.610
495 2024-11-05  14.140
496 2024-11-06  12.670

[497 rows x 2 columns]


In [21]:
# Function to fetch weather data including forecasts
def get_weather_data(polygon_coordinates, days_before_today, forecast_days=3):
    """
    Fetches historical and forecasted weather data and creates shifted features.
    """
    # Calculate date range for historical data
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_before_today)
    start_date_str = start_date.strftime('%Y%m%d')
    end_date_str = end_date.strftime('%Y%m%d')
    
    # Calculate date range for forecast data
    forecast_start_date = end_date + timedelta(days=1)
    forecast_end_date = end_date + timedelta(days=forecast_days)
    forecast_start_str = forecast_start_date.strftime('%Y%m%d')
    forecast_end_str = forecast_end_date.strftime('%Y%m%d')
    
    # Use the centroid of the polygon for the weather API
    lon = np.mean([point[0] for point in polygon_coordinates])
    lat = np.mean([point[1] for point in polygon_coordinates])
    
    # Historical weather data URL
    parameters = "T2M,PRECTOTCORR,ALLSKY_SFC_SW_DWN,RH2M,WS10M,PS"
    nasa_power_url_hist = (
        f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters={parameters}"
        f"&community=AG&longitude={lon}&latitude={lat}"
        f"&start={start_date_str}&end={end_date_str}&format=JSON"
    )
    
    # Fetch historical data
    response_hist = requests.get(nasa_power_url_hist)
    weather_data_hist = response_hist.json()["properties"]["parameter"]
    
    # Forecasted weather data URL
    nasa_power_url_forecast = (
        f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters={parameters}"
        f"&community=AG&longitude={lon}&latitude={lat}"
        f"&start={forecast_start_str}&end={forecast_end_str}&format=JSON"
    )
    
    # Fetch forecasted data
    response_forecast = requests.get(nasa_power_url_forecast)
    weather_data_forecast = response_forecast.json()["properties"]["parameter"]
    
    # Parse historical data into DataFrame
    dates_hist = list(weather_data_hist["T2M"].keys())
    weather_df_hist = pd.DataFrame({
        'date': pd.to_datetime(dates_hist, format='%Y%m%d'),
        'temperature': list(weather_data_hist["T2M"].values()),
        'precipitation': list(weather_data_hist["PRECTOTCORR"].values()),
        'irradiation': list(weather_data_hist["ALLSKY_SFC_SW_DWN"].values()),
        'humidity': list(weather_data_hist["RH2M"].values()),
        'wind_speed': list(weather_data_hist["WS10M"].values()),
        'pressure': list(weather_data_hist["PS"].values())
    })
    
    # Parse forecasted data into DataFrame
    dates_forecast = list(weather_data_forecast["T2M"].keys())
    weather_df_forecast = pd.DataFrame({
        'date': pd.to_datetime(dates_forecast, format='%Y%m%d'),
        'temperature': list(weather_data_forecast["T2M"].values()),
        'precipitation': list(weather_data_forecast["PRECTOTCORR"].values()),
        'irradiation': list(weather_data_forecast["ALLSKY_SFC_SW_DWN"].values()),
        'humidity': list(weather_data_forecast["RH2M"].values()),
        'wind_speed': list(weather_data_forecast["WS10M"].values()),
        'pressure': list(weather_data_forecast["PS"].values())
    })
    
    # Combine historical and forecasted data
    weather_df = pd.concat([weather_df_hist, weather_df_forecast], ignore_index=True)
    
    # Replace placeholder values (-999.00) with NaN
    weather_df.replace(-999.00, np.nan, inplace=True)
    
    # Set 'date' as index
    weather_df.set_index('date', inplace=True)
    
    # Interpolate missing values
    weather_df.sort_index(inplace=True)
    weather_df.interpolate(method='time', inplace=True)
    weather_df.dropna(inplace=True)
    weather_df.reset_index(inplace=True)
    
    # Create shifted features for t+1, t+2, t+3
    for i in range(1, forecast_days + 1):
        weather_df[f'temperature_t+{i}'] = weather_df['temperature'].shift(-i)
        weather_df[f'precipitation_t+{i}'] = weather_df['precipitation'].shift(-i)
        weather_df[f'irradiation_t+{i}'] = weather_df['irradiation'].shift(-i)
        weather_df[f'humidity_t+{i}'] = weather_df['humidity'].shift(-i)
        weather_df[f'wind_speed_t+{i}'] = weather_df['wind_speed'].shift(-i)
        weather_df[f'pressure_t+{i}'] = weather_df['pressure'].shift(-i)
    
    # Remove rows with NaN values resulting from shifting
    weather_df = weather_df.iloc[:-forecast_days]
    return weather_df

# Run the function and print the result
weather_df = get_weather_data(pontelongo_coordinates, days_before_today, forecast_days)
print("Weather Data with Forecast:")
print(weather_df)

Weather Data with Forecast:
          date  temperature  precipitation  irradiation  humidity  wind_speed  \
0   2023-06-28        23.17           0.52        19.97     62.44        4.27   
1   2023-06-29        24.96           0.01        27.67     50.75        2.55   
2   2023-06-30        22.77           3.27        19.29     61.62        1.12   
3   2023-07-01        22.84           4.82        17.67     72.50        1.16   
4   2023-07-02        25.65           0.69        26.10     65.88        1.88   
..         ...          ...            ...          ...       ...         ...   
493 2024-11-02        13.79           0.12         7.78     91.88        2.00   
494 2024-11-03        12.60           0.00         9.20     83.12        2.85   
495 2024-11-04        11.92           0.00         9.20     85.31        1.94   
496 2024-11-05        11.32           0.00         9.20     83.06        2.59   
497 2024-11-06        10.80           0.00         9.20     84.50        2.48   


In [22]:
# Import necessary libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

# Step 1: Retrieve data (Assuming these functions are defined)
# Replace these functions with your actual data retrieval code
sentinel2_df = get_sentinel2_data(pontelongo_coordinates, days_before_today)
soil_related_df = retrieve_soil_moisture_and_related_data(pontelongo_coordinates, days_before_today)
soil_moisture_df = get_soil_moisture_data(pontelongo_coordinates, days_before_today)
lst_df = get_lst_data(pontelongo_coordinates, days_before_today)
weather_df = get_weather_data(pontelongo_coordinates, days_before_today, forecast_days)

# Step 2: Merge dataframes on 'date'
df = soil_moisture_df
df = df.merge(soil_related_df, how='inner', left_index=True, right_index=True)
df = df.merge(sentinel2_df.set_index('date'), how='inner', left_index=True, right_index=True)
df = df.merge(lst_df.set_index('date'), how='inner', left_index=True, right_index=True)
df = df.merge(weather_df.set_index('date'), how='inner', left_index=True, right_index=True)
df.reset_index(inplace=True)

# Step 3: Prepare the data
# Create target variables for soil moisture at t+1, t+2, t+3
forecast_days = 3
for i in range(1, forecast_days + 1):
    df[f'soil_moisture_t+{i}'] = df['soil_moisture'].shift(-i)

# Remove last forecast_days rows with NaN targets
df_full = df.copy()  # Keep a copy before dropping rows
df = df.iloc[:-forecast_days]

# Prepare features and labels
target_columns = [f'soil_moisture_t+{i}' for i in range(1, forecast_days + 1)]
feature_columns = df.columns.drop(['date', 'soil_moisture'] + target_columns)

X = df[feature_columns].values
y = df[target_columns].values

# Normalize features and targets
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler_X = StandardScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Step 4: Split data into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)

# Step 5: Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, output_dim)
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
input_dim = X_train.shape[1]
output_dim = y_train.shape[1]
model = SimpleNN(input_dim, output_dim)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert data to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

# Step 6: Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Step 7: Make predictions for the next 3 days
# Get today's features
features_today = df_full.iloc[-1][feature_columns]
features_today_scaled = scaler_X.transform([features_today.values])
features_today_tensor = torch.tensor(features_today_scaled, dtype=torch.float32)

# Predict soil moisture for the next 3 days
model.eval()
with torch.no_grad():
    predicted_soil_moisture_scaled = model(features_today_tensor)
    predicted_soil_moisture = scaler_y.inverse_transform(predicted_soil_moisture_scaled.numpy())

print("\nPredicted soil moisture for the next 3 days:")
for i in range(forecast_days):
    print(f"Day t+{i + 1}: {predicted_soil_moisture[0][i]:.4f}")

# Step 8: Provide irrigation suggestions
# Define soil moisture threshold
threshold = 0.3  # Adjust based on your specific threshold

# Get forecasted precipitation
today = df_full.iloc[-1]['date']
forecasted_precipitation = weather_df[weather_df['date'] > today].head(forecast_days).reset_index()

# Make suggestions based on soil moisture and precipitation
print("\nIrrigation Suggestions:")
for i in range(forecast_days):
    soil_moisture_value = predicted_soil_moisture[0][i]
    precipitation = forecasted_precipitation.iloc[i]['precipitation']
    if precipitation > 5:
        print(f"Day t+{i + 1}: Forecasted rainfall is {precipitation}mm (>5mm). Suggest to wait.")
    elif soil_moisture_value < threshold:
        print(f"Day t+{i + 1}: Soil moisture ({soil_moisture_value:.4f}) is below threshold and forecasted rainfall is {precipitation}mm. Suggest to irrigate.")
    else:
        print(f"Day t+{i + 1}: Soil moisture ({soil_moisture_value:.4f}) is above threshold and forecasted rainfall is {precipitation}mm. No need to irrigate.")


Number of images retrieved: 97
Epoch [10/100], Loss: 0.1108
Epoch [20/100], Loss: 0.0501
Epoch [30/100], Loss: 0.0267
Epoch [40/100], Loss: 0.0145
Epoch [50/100], Loss: 0.0095
Epoch [60/100], Loss: 0.0071
Epoch [70/100], Loss: 0.0057
Epoch [80/100], Loss: 0.0047
Epoch [90/100], Loss: 0.0040
Epoch [100/100], Loss: 0.0035

Predicted soil moisture for the next 3 days:
Day t+1: 0.2344
Day t+2: 0.2265
Day t+3: 0.1917

Irrigation Suggestions:
Day t+1: Soil moisture (0.2344) is below threshold and forecasted rainfall is 0.0mm. Suggest to irrigate.
Day t+2: Soil moisture (0.2265) is below threshold and forecasted rainfall is 0.0mm. Suggest to irrigate.
Day t+3: Soil moisture (0.1917) is below threshold and forecasted rainfall is 0.0mm. Suggest to irrigate.
